In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from sentence_transformers import SentenceTransformer
import uuid

# Connect to Qdrant on your VPS
client = QdrantClient(
    host="vps.maestri.com.co",
    port=6333,
    https=False
)

collection_name = "maestri_knowledge"
embedding_size = 512  # for distiluse-base-multilingual-cased-v2

# STEP 1: Recreate collection with correct vector size
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=embedding_size, distance=Distance.COSINE),
)

# STEP 2: Load model
model = SentenceTransformer("distiluse-base-multilingual-cased-v2")

# STEP 3: Define business documents
documents = [
    {"category": "contacto", "text": "Horarios: Lunes a Viernes de 10:00 A.M a 6:30 P.M. Sábados y domingos de 10:00 A.M a 6:00 P.M."},
    {"category": "contacto", "text": "Dirección: CRA 27A # 68-87 Bogotá DC Colombia. Email: josue.santamaria@maestri.co. Whatsapp: (1) 7328309 - (+57) 3154831517."},
    {"category": "nosotros", "text": "Maestri Milano es una boutique de vinos y alimentos gourmet que lleva a tu mesa una auténtica experiencia gastronómica a la italiana."},
    {"category": "nosotros", "text": "Desde 2019 producimos charcutería artesanal italiana en Colombia con estándares y recetas milenarias italianas."},
    {"category": "nosotros", "text": "Ofrecemos vinos de más de 40 denominaciones y 80 referencias exclusivas, junto a alimentos como Parmigiano Reggiano, trufa y aceite extra virgen."},
    {"category": "equipo", "text": "Riccardo Trentini – Sales & Finance"},
    {"category": "equipo", "text": "Carlo Angius – Process Optimization & Markets"},
    {"category": "equipo", "text": "Danilo Marotta – Administración y Producción"},
    {"category": "equipo", "text": "Manuel Ghiddi – Maestro Charcutero"},
    {"category": "productos", "text": "📦 [Placeholder] Aquí se agregarán descripciones de productos."},
    {"category": "recetas", "text": "📖 [Placeholder] Aquí se agregarán recetas italianas tradicionales."}
]

# STEP 4: Vectorize and insert into Qdrant
points = []
for doc in documents:
    vector = model.encode(doc["text"], convert_to_numpy=True, normalize_embeddings=True).tolist()
    point_id = str(uuid.uuid4())
    payload = {
        "text": doc["text"],
        "category": doc["category"]
    }
    points.append(PointStruct(id=point_id, vector=vector, payload=payload))

client.upsert(collection_name=collection_name, points=points)

print(f"✅ Inserted {len(points)} documents into Qdrant collection: {collection_name}")


/Users/diegohernandez/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/diegohernandez/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/xq/gclpc2_d3ql92h6tddvfwkjr0000gn/T/ipykernel_1453/2386061255.py:17: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


✅ Inserted 11 documents into Qdrant collection: maestri_knowledge
